## Kautilya Midterm

In [10]:
import selenium
import pandas as pd
from bs4 import BeautifulSoup
import time
import re
import nltk

In [11]:
from selenium.webdriver import Chrome, Firefox
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [12]:
# subject to change to your own driver
# browser = Firefox()
browser = webdriver.Firefox()
url = 'https://arxiv.org/'
filter_word = 'healthcare'
browser.get(url)

## Interaction 

In [13]:
input_search = browser.find_element_by_name('query')
input_search.send_keys(filter_word)
input_search.submit()
time.sleep(0.5)

In [14]:
more_links = browser.find_elements_by_link_text('▽ More')
for more in more_links:
    more.click()

ElementClickInterceptedException: Message: Element <a class="is-size-7"> is not clickable at point (306,886) because another element <div id="pendo-guide-container" class="_pendo-step-container-styles"> obscures it


In [15]:
html=browser.page_source
soup=BeautifulSoup(html,'html.parser')
time.sleep(2)

In [16]:
health_articles = soup.find('ol')
time.sleep(2)

In [18]:
titles = [title.text.strip() for title in health_articles.find_all('p', {'class': 'title'})]

In [19]:
arcxiv_id = [o.find_next('a').text for o in health_articles.find_all('p', {'class': 'list-title'})]

In [20]:
article_links = [o.find('a').attrs['href'] for o in health_articles.find_all('p', {'class': 'list-title'})]

In [21]:
pdf = [o.find_next('span').find_next('a').attrs['href'] for o in health_articles.contents[:-1:2]]

In [22]:
time.sleep(0.5)

In [23]:
auth  = [a.find_all('a') for a in health_articles.find_all('p',{'class':'authors'})]
auth_joined = [', '.join([o.text for o in x]) for x in auth]

In [24]:
abstracts =  [o.text.strip() for o in health_articles.find_all('span',{'class':'abstract-full'})]

In [25]:
date_submitted = [o.text.strip().split(';')[0][10:] for o in health_articles.find_all('p',{'class':'is-size-7'}) if 'Submitted' in o.text.strip().split(';')[0] ]

In [26]:
articles_list =[[titles[i],arcxiv_id[i], article_links[i], auth_joined[i], pdf[i], abstracts[i],date_submitted[i]] for i in range(len(titles))]

In [27]:
df_links_article = pd.DataFrame(articles_list, columns=['Title','Arxiv id', 'Article', 'Authors','PDF', 'Abstract','Submitted'])

In [28]:
df_links_article.head()

,Title,Arxiv id,Article,Authors,PDF,Abstract,Submitted
0,Interpretable Deep Learning for Automatic Diag...,arXiv:2010.10328,https://arxiv.org/abs/2010.10328,"Dongdong Zhang, Xiaohui Yuan, Ping Zhang",https://arxiv.org/pdf/2010.10328,Electrocardiogram (ECG) is a widely used relia...,"20 October, 2020"
1,Leveraging Technology for Healthcare and Retai...,arXiv:2010.10285,https://arxiv.org/abs/2010.10285,"Ayan Chatterjee, Ali Shahaab, Martin W. Gerdes...",https://arxiv.org/pdf/2010.10285,Health data is a sensitive category of persona...,"20 October, 2020"
2,Towards new forms of particle sensing and mani...,arXiv:2010.10269,https://arxiv.org/abs/2010.10269,"Jinlong Zhu, Ni Zhao, Renjie Zhou",https://arxiv.org/pdf/2010.10269,Close to half of the world population have sma...,"8 September, 2020"
3,University Operations During a Pandemic: A Fle...,arXiv:2010.10112,https://arxiv.org/abs/2010.10112,"Himanshu Kharkwal, Dakota Olson, Jiali Huang, ...",https://arxiv.org/pdf/2010.10112,Modeling infection spread during pandemics is ...,"20 October, 2020"
4,The Role of Robotics in Infectious Disease Crises,arXiv:2010.09909,https://arxiv.org/abs/2010.09909,"Gregory Hager, Vijay Kumar, Robin Murphy, Dani...",https://arxiv.org/pdf/2010.09909,The recent coronavirus pandemic has highlighte...,"19 October, 2020"


In [29]:
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

def clean(text):
    text=re.sub(r'[^\w\s]',' ',text).lower()   
    words=re.split(r'\W+', text)                   
    non_stop_words=[word for word in words if word not in stopword]        
    lem_words= [wn.lemmatize(word) for word in non_stop_words]             
    clean_text= ' '.join(lem_words)
    return clean_text

In [30]:
df_links_article['clean_text'] = df_links_article['Abstract'].apply(clean)

In [31]:
df_links_article.head()

,Title,Arxiv id,Article,Authors,PDF,Abstract,Submitted,clean_text
0,Interpretable Deep Learning for Automatic Diag...,arXiv:2010.10328,https://arxiv.org/abs/2010.10328,"Dongdong Zhang, Xiaohui Yuan, Ping Zhang",https://arxiv.org/pdf/2010.10328,Electrocardiogram (ECG) is a widely used relia...,"20 October, 2020",electrocardiogram ecg widely used reliable non...
1,Leveraging Technology for Healthcare and Retai...,arXiv:2010.10285,https://arxiv.org/abs/2010.10285,"Ayan Chatterjee, Ali Shahaab, Martin W. Gerdes...",https://arxiv.org/pdf/2010.10285,Health data is a sensitive category of persona...,"20 October, 2020",health data sensitive category personal data m...
2,Towards new forms of particle sensing and mani...,arXiv:2010.10269,https://arxiv.org/abs/2010.10269,"Jinlong Zhu, Ni Zhao, Renjie Zhou",https://arxiv.org/pdf/2010.10269,Close to half of the world population have sma...,"8 September, 2020",close half world population smartphones typica...
3,University Operations During a Pandemic: A Fle...,arXiv:2010.10112,https://arxiv.org/abs/2010.10112,"Himanshu Kharkwal, Dakota Olson, Jiali Huang, ...",https://arxiv.org/pdf/2010.10112,Modeling infection spread during pandemics is ...,"20 October, 2020",modeling infection spread pandemic new model u...
4,The Role of Robotics in Infectious Disease Crises,arXiv:2010.09909,https://arxiv.org/abs/2010.09909,"Gregory Hager, Vijay Kumar, Robin Murphy, Dani...",https://arxiv.org/pdf/2010.09909,The recent coronavirus pandemic has highlighte...,"19 October, 2020",recent coronavirus pandemic highlighted many c...


In [32]:
count_vect=CountVectorizer()

In [33]:
X_counts = count_vect.fit_transform(df_links_article['clean_text'])

In [34]:
vocab=count_vect.get_feature_names()

In [35]:
vectors_model=pd.DataFrame(X_counts.toarray(), columns=vocab)

In [36]:
vectors_model

,000,03,05,10,100,109,11ax,12,13,14,...,worker,workflow,workshop,world,worldwide,would,wound,year,yet,zeroth
0,0,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,7,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
vectors_model.filter('natural language proccesing')

""
0
1
2
3
4
5
6
7
8
9


In [39]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_links_article['clean_text'])
features = tfidf_vectorizer.get_feature_names()

In [47]:
tf_idf_model = pd.DataFrame(tfidf_matrix.toarray(), columns=features)
tf_idf_model

,000,03,05,10,100,109,11ax,12,13,14,...,worker,workflow,workshop,world,worldwide,would,wound,year,yet,zeroth
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.254371,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.121445,0.000000,0.066847,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.063488,0.000000,0.000000,0.000000,0.000000,0.078915,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.066636,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054587,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.064189,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.069397,0.000000,0.153476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.057360,0.000000,0.063145,0.499095,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [54]:
nbrs = NearestNeighbors(n_neighbors=10).fit(tfidf_matrix)

def get_close_neighbours(name):
    row = df_links_article.iloc(name)
    distances, indices = nbrs.kneighbors(tfidf_matrix.getrow(row))
    similar_names = pd.Series(indices.flatten()).map(df_links_article.reset_index()['clean_text'])
    result = pd.DataFrame({'distance':distances.flatten(), 'Abstract':similar_names})
    return result

In [55]:
get_close_neighbours('natural language processing')

ValueError: No axis named natural language processing for object type DataFrame